In [20]:
# Dash framework and core components
import dash
from dash import dcc, html, ctx
from dash.dependencies import Input, Output
import dash_player

# Dash Bootstrap Components for styling
import dash_bootstrap_components as dbc

# Plotly for interactive plotting
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# Pandas for data manipulation
import pandas as pd

# Additional Python libraries
import time
from datetime import timedelta


In [21]:
df = pd.read_csv('../data/testdata/exampleFile.csv')
labelListDF = pd.read_csv('../data/testdata/testLabelList.csv')

In [22]:
cols = list(df.columns)
# if df has a confidence column, remove it from column list
if "confidence" in cols:
    pass
    # cols.remove('confidence')
# else, make a confidence column assigned an undefined
else:
    df['confidence'] = "Undefined"

# if df has a label column, remove it from column list
if "label" in cols:
    pass
    # cols.remove('label')
# else, make a label column assigned an undefined
else:
    df['label'] = "Undefined"

In [23]:
drive_link = 'https://drive.google.com/uc?id=1LmJ3fRtpX-vmrzEaXEUzlPpErXBhr6tB'
cols = ["accel_x"]

In [24]:
colorList = ('#4363d8', '#e6194b', '#3cb44b', '#ffe119', '#f58231', '#911eb4', '#46f0f0', '#f032e6', '#bcf60c', '#fabebe',
             '#008080', '#e6beff', '#9a6324', '#fffac8', '#800000', '#aaffc3', '#808000', '#ffd8b1', '#000075', '#808080',
             '#000000')
# This assigns a specific color from colorList to each label in provided label list ** IF LABEL IS UNDEFINED COLOR IS BLACK**
colorDict = {label: ('#000000' if label == 'Undefined' else color) for label, color in zip(labelListDF, colorList)}

# Confidence list & dictionary
confidenceValues = ("High", "Medium", "Low", "Undefined")
confidenceColors = ('#3cb44b', '#ffe119', '#FF0000', '#000000')
confDict = dict(zip(confidenceValues, confidenceColors))

## Original Working (commented out retrieving COLS from other cell)

In [25]:
#@title Manual Labeling Interface
variable_name = ""
labelList = list(labelListDF)
size = len(df)

# get user input data from cols interface
#cols = handle_submit(submit_button)

# set layout
fig_layout = make_subplots(rows=3, cols=1,
                    shared_xaxes=True,
                    vertical_spacing=0.1,
                    row_heights=[5,5,50],
                    subplot_titles=("Label Line", "Confidence Line", "Raw Time-Series Data"),
                    )
fig_layout.update_layout(height=10, width=1000)
# link x-axis for timeseries slider
fig_layout.update_xaxes(matches='x')
# theme stuff
#load_figure_template("bootstrap")
#fig_layout.update_layout(template="bootstrap")

# create UI fields for manually adding label & confidence level
manual_label_UI_fields = dbc.Card(
    [
      html.H4(children='''User Input Fields for Manually Adding a Label:'''),
      html.Div(children='''
        Format as YYYY-MM-DD HH:MM:SS E.g (2021-10-08 16:50:21)
      '''),
      html.Br(),
      
      html.Div([
          "Start Time: ",
          # Need to change ids, etc
          dcc.Input(id='start-input', 
                    type='text', 
                    placeholder = "YYYY-MM-DD HH:MM:SS"),
          dcc.Checklist(id='start-checkbox',
                    options=[{'label': 'Fill Start UI with Click', 'value': 'Checked'}],
                    value=['Unchecked']),
          html.Div(id='start-output'), 
          html.Div(id='start-cb-output'),
      ]),
      html.Br(),
      
      html.Div([
          "End   Time: ",
          # Need to change ids, etc
          dcc.Input(id='end-input',
                    type='text',
                    placeholder = "YYYY-MM-DD HH:MM:SS"),
          dcc.Checklist(id='end-checkbox',
                    options=[{'label': 'Fill End UI with Click', 'value': 'Checked'}],
                    value=['Unchecked']),
          html.Div(id='end-output'), 
          html.Div(id='end-cb-output'),
      ]),
      html.Br(),
      
      html.Div([
          "Labels: ",
          # Dummy values, need to get values from labelList csv
          dcc.Dropdown(labelList, 
                      placeholder = 'Select a Label',
                      id='label-selection'),
          # https://dash.plotly.com/dash-core-components/dropdown
          # this was for an output call back that prints curr value
          html.Div(id='label-output')
      ]),
      html.Br(),
      
      html.Div([
          "Degree of Confidence: ",
          dcc.Dropdown(["High", "Medium", "Low", "Undefined"], 
                      placeholder = 'Select a Confidence Level',
                      id='confidence-selection'),
          html.Div(id='confidence-output')
      ]),
      html.Br(),
      
      html.Button('Update Graph', id='btn-manual-label', n_clicks=0),
      html.Br(),
    ],
    body=True,
)

# create UI fields for video operations
video_UI_fields = dbc.Card(
    [
        # input field for offset
        html.H5(children='''Set Data/Video Offset:'''),
        html.Div(children='''Zero Offset: Both video and data start at zero.'''),
        html.Br(),
        html.Div(children='''Positive Offset: Data starts at zero, and video starts at offset."'''),
        html.Br(),
        html.Div(children='''Negative Offset: Data starts at offset, and video starts at zero."'''),
        html.Br(),
        dcc.Input(id='video-offset-input', type='text', placeholder = "Input in Seconds", value="0"),
        html.Div(id='vid-offset-output'),
        html.Br(),

        # sync video -> data
        html.H5(children='''Sync Video to Data:'''),
        html.Div(children='''Plot line on data graph at current time in video.'''),
        html.Button("Sync", id="button-sync-vid", n_clicks=0),
        html.Br(), html.Br(),


        # seek time input field
        html.H5(children='''Seek to Time in Video:'''),
        html.Div(children='''E.g. If you want to go to 2 minute time stamp in video, 
          input in seconds, or "120"'''),
        dcc.Input(id='seek-input', type='text', placeholder = "Input in Seconds"),
        html.Button("Seek", id="button-seek-to"),
        html.Div(id="div-current-time"),
        html.Div(html.Button('Reset Inputs', id='reset-button-2')),
        ],
    body=True,
)

# Makes list and assigns highest value to each index
labelLine = []
for i in range(size):
    labelLine.append(1)

def plotGraph():
    # Make figure for raw time series
    i = 0
    for element in cols:
      fig_layout.add_trace(go.Scatter(x=df['datetime'], y=df[cols[i]],
                      mode='lines', # 'lines' or 'markers'
                      name=cols[i]), row=3, col=1)
      i+=1
    fig_layout.update_layout(
        xaxis=dict(),#end xaxis  definition
        xaxis1_rangeslider_visible=False,
        xaxis2_rangeslider_visible=False,
        xaxis3_rangeslider_visible=True
        )
    # Count number of labels
    tempDf = df.ne(df.shift())
    labelCount = tempDf.loc[tempDf.label == True, 'label'].count()
    # pd.set_option('display.max_rows', None)  # or 1000
    # print(tempDf['label'])
    # print(labelCount)

    # Make two lists to store indices
    labelsStartIndex = []
    labelsEndIndex = []
    # Start index at -1 to match indices
    index = -1
    # Take size to put into endIndex array to show last index
    size = len(tempDf['label'])

    # Loop through column to find Start Indices
    for element in tempDf['label']:
        if element == True:
            index+=1
            labelsStartIndex.append(index)
        else:
            index+=1
    # print("Start Indices", labelsStartIndex)

    # Loop through Start Indices list and get the index before next change
    for element in labelsStartIndex:
        labelsEndIndex.append(element - 1)
    # Remove first so we dont get the garbage value
    labelsEndIndex.pop(0)
    # Append size of column because thats last known label index
    labelsEndIndex.append(size)
    # print("End Indices", labelsEndIndex)

    # line with labels
    i = 0
    size = len(labelsStartIndex)
    size = int(size)
    currLabel = df['label']
    for x in range(size):
        fig_layout.add_trace(go.Scatter(
            x = df.loc[labelsStartIndex[i]:labelsEndIndex[i],'datetime'],
            y = labelLine,
            mode="lines",
            name=currLabel.at[labelsStartIndex[i]],
            text=df.loc[labelsStartIndex[i]:labelsEndIndex[i],'label'],
            line_color=colorDict[currLabel.at[labelsStartIndex[i]]],
            textposition="top center",
            line_width=5,  # set the line width to 2
            showlegend=False
            ), row=1, col=1)
        i+=1

    # line with degree of confidence
    i = 0
    currConfidence = df['confidence']
    for x in range(size):
        fig_layout.add_trace(go.Scatter(
            x = df.loc[labelsStartIndex[i]:labelsEndIndex[i],'datetime'],
            y = labelLine,
            mode="lines",
            name=currConfidence.at[labelsStartIndex[i]],
            text=df.loc[labelsStartIndex[i]:labelsEndIndex[i],'confidence'],
            line_color=confDict[currConfidence.at[labelsStartIndex[i]]],
            textposition="top center",
            line_width=5,  # set the line width to 2
            showlegend=False
            ), row=2, col=1)
        i+=1

    fig_layout.update_layout(height=500, #width=1000,
                  legend_title_text="Sensors")
    fl_reset = go.Figure(fig_layout)
    # Reference website https://dash.plotly.com/layout
    # Build App
    external_stylesheets = [dbc.themes.BOOTSTRAP]
    #load_figure_template("bootstrap")

    app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
    app.title = 'Manually Label Time-Series Data'
    
    app.layout = dbc.Container(
      [
          html.H2("Manually Label Raw Time-Series Data"),
          html.Hr(),
          # html.Div(children='''
          #   Dash: A web application framework for your data.
          # '''),
          html.Br(),
          
          # data
          dbc.Row(
              [
                  dbc.Col(manual_label_UI_fields, md=4),
                  # dbc.Col(dcc.Graph(id='graph-output',figure={}), md=8),
                  dbc.Col(dbc.Card([
                      dcc.Graph(id='graph-output',figure={})
                  ]), 
                  md=8),
              ],
              align="center", style={"height": "rem"}
            ),
       
          # video
          dbc.Row(
              [
                  dbc.Col(video_UI_fields, md=4),
                  dbc.Col(dbc.Card([
                      dash_player.DashPlayer(id = "video-player", url=drive_link, controls=True)
                  ])
                  , md=4),
                  # dbc.Col(dash_player.DashPlayer(id = "video-player", url=url, controls=True), md=8),
                  html.Br(),
                  # current vid vals
                  html.Hr(),
                  html.Div(id='vid-sync-plot-dt-output'),
                  html.Div(id='vid-sync-plot-offset-output'),
                  html.Div(id='vid-sync-plot-timestamp-output'),               
              ],
              align="center",
          ),
      ],
      fluid=True,
    )
    
    #####################################################
    # callback for user input start time
    @app.callback(
        Output("start-output", "children"),
        Input('start-input', "value")
    )
    def update_start_time(start_value):
        update_start_time.data = start_value
        return "Start Value: {}".format(start_value)
    
    # callback for user input end time
    @app.callback(
        Output("end-output", "children"),
        Input('end-input', "value")
    )
    def update_end_time(end_value):
        update_end_time.data = end_value
        return "End Value: {}".format(end_value)
    
    # callback for user input label selection
    @app.callback(
        Output("label-output", "children"),
        Input('label-selection', "value")
    )
    def update_label(label_value):
        update_label.data = label_value
        return "Label Selection: {}".format(label_value)
    
    # callback for user input confidence selection
    @app.callback(
        Output("confidence-output", "children"),
        Input('confidence-selection', "value")
    )
    def update_confidence_degree(confidence_value):
        update_confidence_degree.data = confidence_value
        return "Degree of Confidence: {}".format(confidence_value)
    
    # callback for user input video offset
    @app.callback(
        Output("vid-offset-output", "children"),
        Input('video-offset-input', "value")
    )
    def update_video_offset(videoOffset):
        update_video_offset.data = videoOffset
        return "Offset Value: {}".format(videoOffset)

    # callback for printing current time under video
    @app.callback(
        Output("div-current-time", "children"),
        Input("video-player", "currentTime")
    )
    def update_time(currentTime):
        update_time.data = currentTime
        return "Current Timestamp of Video: {}".format(currentTime)

    @app.callback(
        Output('graph-output', 'figure'),
        [Input('btn-manual-label', 'n_clicks'),
        Input('button-sync-vid', 'n_clicks'),]
    )
    def updateGraph(btn1, btn2):
        # Initial render of graph
        # What we update when adding new labels
        updatedFig = fl_reset
        vid_to_data_sync = 0
        offset = 0
        timestamp = 0
        # If btn is clicked
        if "btn-manual-label" == ctx.triggered_id:
          start = update_start_time.data
          end = update_end_time.data
          label = update_label.data
          confidence = update_confidence_degree.data

          # Initialize a copy of df to manipulate
          label_df = df.copy()

          # Removes any decimals in datetime column, can assume label last at least a second
          # 2021-10-08 16:50:21.000000000 -> 16:50:21
          label_df['datetime'] = pd.to_datetime(label_df['datetime']).dt.strftime('%Y-%m-%d %H:%M:%S')

          # using strf converts datetime to string
          # using strp converts string to datetime
          """
          start = datetime.strftime(start, '%Y-%m-%d %H:%M:%S')
          end = datetime.strftime(end, '%Y-%m-%d %H:%M:%S')
          print(type(start), type(end))
          """
          
          # find start & end within 'datetime' column
          # Find start and end within 'datetime' column
          # Get Start indices and shove into list, First indice is where first label needs to go
          startIndices = label_df.index[label_df['datetime'] == start].tolist()
          startIndex = startIndices[0]
          # print("Starting Index", startIndex)
          # Get End indices and shove into list, Last indice is where labels need to end
          endIndices = label_df.index[label_df['datetime'] == end].tolist()
          endIndex = endIndices[-1]
          # print("Ending Index", endIndex)
          
          # Within df, look at 'label' column & assign label to range of startIndex to endIndex
          label_df.loc[startIndex:endIndex, 'label'] = label

          # label_df.label has our updated label column
          # Replaces original label column with the newly updated label column
          df.loc[:, 'label'] = label_df.label
          
          # replace confidence value inbetween start and end index
          label_df.loc[startIndex:endIndex, 'confidence'] = confidence
          df.loc[:, 'confidence'] = label_df.confidence
          # Check updated original dataframe's label column to see if updated
          # print(df['label'])
          
          # **** LABEL LINE **** #

          # Count number of labels
          tempDf = df.ne(df.shift())
          labelCount = tempDf.loc[tempDf.label == True, 'label'].count()
          # pd.set_option('display.max_rows', None)  # or 1000
          # print(tempDf['label'])
          # print(labelCount)

          # Make two lists to store indices
          labelsStartIndex = []
          labelsEndIndex = []
          # Start index at -1 to match indices
          index = -1
          # Take size to put into endIndex array to show last index
          size = len(tempDf['label'])

          # Loop through column to find Start Indices
          for element in tempDf['label']:
              if element == True:
                  index+=1
                  labelsStartIndex.append(index)
              else:
                  index+=1
          # print("Start Indices", labelsStartIndex)

          # Loop through Start Indices list and get the index before next change
          for element in labelsStartIndex:
              labelsEndIndex.append(element - 1)
          # Remove first so we dont get the garbage value
          labelsEndIndex.pop(0)
          # Append size of column because thats last known label index
          labelsEndIndex.append(size)
          # print("End Indices", labelsEndIndex)

          # Pandas iloc or whatever to plot between datetime ranges [0,31] on first iteration of loop
          # line with labels
          i = 0
          size = len(labelsStartIndex)
          size = int(size)
          currLabel = df['label']
          
          # Update the graph
          updatedFig = go.Figure(fl_reset)
          
          # plot label line
          for x in range(size):
              updatedFig.add_trace(go.Scatter(
                  x = df.loc[labelsStartIndex[i]:labelsEndIndex[i],'datetime'],
                  y = labelLine,
                  mode="lines",
                  name=currLabel.at[labelsStartIndex[i]],
                  text=df.loc[labelsStartIndex[i]:labelsEndIndex[i],'label'],
                  line_color=colorDict[currLabel.at[labelsStartIndex[i]]],
                  textposition="top center",
                  line_width=5,  # set the line width to 2
                  showlegend=False
                  ), row=1, col=1)
              i+=1

          # line with degree of confidence
          i = 0
          currConfidence = df['confidence']
          for x in range(size):
              updatedFig.add_trace(go.Scatter(
                  x = df.loc[labelsStartIndex[i]:labelsEndIndex[i],'datetime'],
                  y = labelLine,
                  mode="lines",
                  name=currConfidence.at[labelsStartIndex[i]],
                  text=df.loc[labelsStartIndex[i]:labelsEndIndex[i],'confidence'],
                  line_color=confDict[currConfidence.at[labelsStartIndex[i]]],
                  textposition="top center",
                  line_width=5,  # set the line width to 2
                  showlegend=False
                  ), row=2, col=1)
              i+=1

        if "button-sync-vid" == ctx.triggered_id:
            # updatedFig = fig

            # # create copy df to modify
            temp_df = df.copy()

            # Assume all datetime are in column named 'datetime'
            # Take first & last datetime from column
            firstDateTime = temp_df['datetime'][0]
            lastDateTime = temp_df['datetime'][len(temp_df['datetime']) - 1]

            firstDateTime = pd.to_datetime(firstDateTime)
            lastDateTime = pd.to_datetime(lastDateTime)

            # Convert first & last values to datetime, w/ unix specifications
            # cut down decimal
            firstDateTime = firstDateTime.strftime('%Y-%m-%d %H:%M:%S')
            # have to convert back to datetime
            firstDateTime = pd.to_datetime(firstDateTime)
            # unix specifications
            firstDateTime = pd.to_datetime(firstDateTime, unit='s', origin='unix')
            # cut down decimal
            lastDateTime = lastDateTime.strftime('%Y-%m-%d %H:%M:%S')
             # have to convert back to datetime
            lastDateTime = pd.to_datetime(lastDateTime)
            # unix specifications
            lastDateTime = pd.to_datetime(lastDateTime, unit='s', origin='unix')

            # Get unix time for first and last (datetime -> unix)
            firstUnixTime = (time.mktime(firstDateTime.timetuple()))
            lastUnixTime = (time.mktime(lastDateTime.timetuple()))

            # offset & timestamp retrieval 
            # get from update_video_offset.data, strp by casting as int
            offset = int(update_video_offset.data)
            # get from update_time.data, , strp by casting as int
            timestamp = int(update_time.data)

            # if our offset is negative
            # be able to start data before its recorded
            if offset < 0:
              vid_to_data_sync = firstUnixTime + timestamp + abs(offset)

            else:
              # # get actual time after math in unix
              vid_to_data_sync = firstUnixTime + timestamp - offset
            # # # unix -> datetime, and strf down to whole second
            vid_to_data_sync = pd.to_datetime(vid_to_data_sync, unit='s', origin='unix')
            vid_to_data_sync = vid_to_data_sync.strftime('%Y-%m-%d %H:%M:%S')

            # reset graph
            updatedFig = go.Figure(fl_reset)

            # plot at that x a red line
            updatedFig.add_vline(
              # hard coded range to highlight, change these x's to video time
              x=vid_to_data_sync, line_width=2, line_dash="dash", line_color="red"
            )
            # label added at same x value
            updatedFig.add_annotation(x=vid_to_data_sync, text="Video Sync", font=dict(size=15, color="black"),align="center")
              
        return updatedFig
    
    # callback for seeking to timestamp in video from user inputted value
    @app.callback(
        Output("video-player", "seekTo"), 
        [Input("button-seek-to", "n_clicks"),
        Input('seek-input', 'value')]
    )
    def set_seekTo(n_clicks, seek_value):
        if 'button-seek-to' == ctx.triggered_id:
          return seek_value

    # callback for resetting video UI
    @app.callback(
        [Output('video-offset-input', 'value'),Output('seek-input', 'value')],
        [Input('reset-button-2', 'n_clicks')])
    def reset_input(n_clicks):
        if n_clicks:
            return '0','0'
        else:
            return '0','0'
    
    # checkbox
    @app.callback(Output('start-cb-output', 'children'), [Input('start-checkbox', 'value')])
    def start_ui_fill(value):
        if len(value) == 2:
            start_ui_fill.data = value
            return "Start time fill toggled on"
        else:
            start_ui_fill.data = value
            return "Start time fill toggled off"

    # fill start UI with clicked data
    @app.callback(
        Output('start-input', 'value'),
        Input('graph-output', 'clickData'))
    def st_x_value(clickData):
        try:
          if start_ui_fill.data.count('Checked') == 1:
            xval1 = clickData['points'][0]['x']
            st_x_value.data = xval1
            dt_val1 = xval1.split(".")[0]
            return str(dt_val1)
          else:
            return update_start_time.data
        except Exception as e:
          pass

    # checkbox
    @app.callback(Output('end-cb-output', 'children'), [Input('end-checkbox', 'value')])
    def end_ui_fill(value):
        if len(value) == 2:
            end_ui_fill.data = value
            return "End time fill toggled on"
        else:
            end_ui_fill.data = value
            return "End time fill toggled off"

    # fill start UI with clicked data
    @app.callback(
        Output('end-input', 'value'),
        Input('graph-output', 'clickData'))
    def ed_x_value(clickData):
        try:
          if end_ui_fill.data.count('Checked') == 1:
            xval2 = clickData['points'][0]['x']
            ed_x_value.data = xval2
            dt_val2 = xval2.split(".")[0]
            return str(dt_val2)
          else:
            return update_end_time.data
        except Exception as e:
          pass

    app.run_server(jupyter_mode='external')
plotGraph()

Dash app running on http://127.0.0.1:8050/


## new

### HTML STUFF

In [26]:
# create UI fields for manually adding label & confidence level
manual_label_UI_fields = dbc.Card(
    [
      html.H4(children='''User Input Fields for Manually Adding a Label:'''),
      html.Div(children='''
        Format as YYYY-MM-DD HH:MM:SS E.g (2021-10-08 16:50:21)
      '''),
      html.Br(),
      
      html.Div([
          "Start Time: ",
          # Need to change ids, etc
          dcc.Input(id='start-input', 
                    type='text', 
                    placeholder = "YYYY-MM-DD HH:MM:SS"),
          dcc.Checklist(id='start-checkbox',
                    options=[{'label': 'Fill Start UI with Click', 'value': 'Checked'}],
                    value=['Unchecked']),
          html.Div(id='start-output'), 
          html.Div(id='start-cb-output'),
      ]),
      html.Br(),
      
      html.Div([
          "End   Time: ",
          # Need to change ids, etc
          dcc.Input(id='end-input',
                    type='text',
                    placeholder = "YYYY-MM-DD HH:MM:SS"),
          dcc.Checklist(id='end-checkbox',
                    options=[{'label': 'Fill End UI with Click', 'value': 'Checked'}],
                    value=['Unchecked']),
          html.Div(id='end-output'), 
          html.Div(id='end-cb-output'),
      ]),
      html.Br(),
      
      html.Div([
          "Labels: ",
          # Dummy values, need to get values from labelList csv
          dcc.Dropdown(labelList, 
                      placeholder = 'Select a Label',
                      id='label-selection'),
          # https://dash.plotly.com/dash-core-components/dropdown
          # this was for an output call back that prints curr value
          html.Div(id='label-output')
      ]),
      html.Br(),
      
      html.Div([
          "Degree of Confidence: ",
          dcc.Dropdown(["High", "Medium", "Low", "Undefined"], 
                      placeholder = 'Select a Confidence Level',
                      id='confidence-selection'),
          html.Div(id='confidence-output')
      ]),
      html.Br(),
      
      html.Button('Update Graph', id='btn-manual-label', n_clicks=0),
      html.Br(),
    ],
    body=True,
)

# create UI fields for video operations
video_UI_fields = dbc.Card(
    [
        # input field for offset
        html.H5(children='''Set Data/Video Offset:'''),
        html.Div(children='''Zero Offset: Both video and data start at zero.'''),
        html.Br(),
        html.Div(children='''Positive Offset: Data starts at zero, and video starts at offset."'''),
        html.Br(),
        html.Div(children='''Negative Offset: Data starts at offset, and video starts at zero."'''),
        html.Br(),
        dcc.Input(id='video-offset-input', type='text', placeholder = "Input in Seconds", value="0"),
        html.Div(id='vid-offset-output'),
        html.Br(),

        # sync video -> data
        html.H5(children='''Sync Video to Data:'''),
        html.Div(children='''Plot line on data graph at current time in video.'''),
        html.Button("Sync", id="button-sync-vid", n_clicks=0),
        html.Br(), html.Br(),


        # seek time input field
        html.H5(children='''Seek to Time in Video:'''),
        html.Div(children='''E.g. If you want to go to 2 minute time stamp in video, 
          input in seconds, or "120"'''),
        dcc.Input(id='seek-input', type='text', placeholder = "Input in Seconds"),
        html.Button("Seek", id="button-seek-to"),
        html.Div(id="div-current-time"),
        html.Div(html.Button('Reset Inputs', id='reset-button-2')),
        ],
    body=True,
)

### Actual code

In [28]:
#@title Manual Labeling Interface
variable_name = ""
labelList = list(labelListDF)
size = len(df)

# get user input data from cols interface
#cols = handle_submit(submit_button)

# set layout
fig_layout = make_subplots(rows=3, cols=1,
                    shared_xaxes=True,
                    vertical_spacing=0.1,
                    row_heights=[5,5,50],
                    subplot_titles=("Label Line", "Confidence Line", "Raw Time-Series Data"),
                    )
fig_layout.update_layout(height=10, width=1000)
# link x-axis for timeseries slider
fig_layout.update_xaxes(matches='x')
# theme stuff
#load_figure_template("bootstrap")
#fig_layout.update_layout(template="bootstrap")

# Makes list and assigns highest value to each index
labelLine = []
for i in range(size):
    labelLine.append(1)

# Retrieve start/end indices of all labels
def calculate_label_indices(df):
    tempDf = df.ne(df.shift())
    labelsStartIndex, labelsEndIndex = [], []
    index = -1
    for element in tempDf['label']:
        index += 1
        if element == True:
            labelsStartIndex.append(index)
            if labelsStartIndex and index > 0:
                labelsEndIndex.append(index - 1)
    labelsEndIndex.append(len(tempDf['label']) - 1)
    return labelsStartIndex, labelsEndIndex

labelsStartIndex, labelsEndIndex = calculate_label_indices(df)

def add_trace_to_layout(fig_layout, df, col_name, row, col):
    fig_layout.add_trace(go.Scatter(
        x=df['datetime'], y=df[col_name],
        mode='lines',
        name=col_name), row=row, col=col)

def update_label_lines(fig_layout, labelsStartIndex, labelsEndIndex, df):
    for i in range(len(labelsStartIndex)):
        start_idx = labelsStartIndex[i]
        end_idx = labelsEndIndex[i]
        label = df['label'].iloc[start_idx]

        fig_layout.add_trace(go.Scatter(
            x = df.loc[start_idx:end_idx,'datetime'],
            y = [1] * (end_idx - start_idx + 1),  # Assuming labelLine is a constant value of 1
            mode="lines",
            name=label,
            text=label,
            line_color=colorDict[label],
            textposition="top center",
            line_width=5,
            showlegend=False
        ), row=1, col=1)

def update_confidence_lines(fig_layout, labelsStartIndex, labelsEndIndex, df):
    for i in range(len(labelsStartIndex)):
        start_idx = labelsStartIndex[i]
        end_idx = labelsEndIndex[i]
        confidence = df['confidence'].iloc[start_idx]

        fig_layout.add_trace(go.Scatter(
            x = df.loc[start_idx:end_idx,'datetime'],
            y = [1] * (end_idx - start_idx + 1),  # Assuming labelLine is a constant value of 1
            mode="lines",
            name=confidence,
            text=confidence,
            line_color=confDict[confidence],
            textposition="top center",
            line_width=5,
            showlegend=False
        ), row=2, col=1)


def plotGraph():
    global fig_layout
    # Create a new figure layout for each call of plotGraph
    fig_layout = make_subplots(
        rows=3, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.1,
        row_heights=[5,5,50],
        subplot_titles=("Label Line", "Confidence Line", "Raw Time-Series Data")
    )

    # Add raw time-series data to the plot
    for col in cols:
        add_trace_to_layout(fig_layout, df, col, 3, 1)

    # Update label and confidence lines
    labelsStartIndex, labelsEndIndex = calculate_label_indices(df)
    update_label_lines(fig_layout, labelsStartIndex, labelsEndIndex, df)
    update_confidence_lines(fig_layout, labelsStartIndex, labelsEndIndex, df)

    # Update layout and return the figure
    fig_layout.update_layout(
        xaxis=dict(),  # end xaxis definition
        xaxis1_rangeslider_visible=False,
        xaxis2_rangeslider_visible=False,
        xaxis3_rangeslider_visible=True,
        height=500,  # width=1000,
        legend_title_text="Sensors"
    )
    fig_layout.update_layout(height=500, #width=1000,
                  legend_title_text="Sensors")
    
    return fig_layout

# Set initial figure
initial_figure = plotGraph()

#fl_reset = go.Figure(fig_layout)

# Reference website https://dash.plotly.com/layout
# Build App
external_stylesheets = [dbc.themes.BOOTSTRAP]
#load_figure_template("bootstrap")

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
app.title = 'Manually Label Time-Series Data'

app.layout = dbc.Container(
    [
        html.H2("Manually Label Raw Time-Series Data"),
        html.Hr(),
        # html.Div(children='''
        #   Dash: A web application framework for your data.
        # '''),
        html.Br(),
        
        # data
        dbc.Row(
            [
                dbc.Col(manual_label_UI_fields, md=4),
                # dbc.Col(dcc.Graph(id='graph-output',figure={}), md=8),
                dbc.Col(dbc.Card([
                    dcc.Graph(id='graph-output', figure=initial_figure)
                ]), 
                md=8),
            ],
            align="center", style={"height": "rem"}
        ),
    
        # video
        dbc.Row(
            [
                dbc.Col(video_UI_fields, md=4),
                dbc.Col(dbc.Card([
                    dash_player.DashPlayer(id = "video-player", url=drive_link, controls=True)
                ])
                , md=4),
                # dbc.Col(dash_player.DashPlayer(id = "video-player", url=url, controls=True), md=8),
                html.Br(),
                # current vid vals
                html.Hr(),
                html.Div(id='vid-sync-plot-dt-output'),
                html.Div(id='vid-sync-plot-offset-output'),
                html.Div(id='vid-sync-plot-timestamp-output'),               
            ],
            align="center",
        ),
    ],
    fluid=True,
)

#####################################################
# callback for user input start time
@app.callback(
    Output("start-output", "children"),
    Input('start-input', "value")
)
def update_start_time(start_value):
    update_start_time.data = start_value
    return "Start Value: {}".format(start_value)

# callback for user input end time
@app.callback(
    Output("end-output", "children"),
    Input('end-input', "value")
)
def update_end_time(end_value):
    update_end_time.data = end_value
    return "End Value: {}".format(end_value)

# callback for user input label selection
@app.callback(
    Output("label-output", "children"),
    Input('label-selection', "value")
)
def update_label(label_value):
    update_label.data = label_value
    return "Label Selection: {}".format(label_value)

# callback for user input confidence selection
@app.callback(
    Output("confidence-output", "children"),
    Input('confidence-selection', "value")
)
def update_confidence_degree(confidence_value):
    update_confidence_degree.data = confidence_value
    return "Degree of Confidence: {}".format(confidence_value)

# callback for user input video offset
@app.callback(
    Output("vid-offset-output", "children"),
    Input('video-offset-input', "value")
)
def update_video_offset(videoOffset):
    update_video_offset.data = videoOffset
    return "Offset Value: {}".format(videoOffset)

# callback for printing current time under video
@app.callback(
    Output("div-current-time", "children"),
    Input("video-player", "currentTime")
)
def update_time(currentTime):
    update_time.data = currentTime
    return "Current Timestamp of Video: {}".format(currentTime)

@app.callback(
    Output('graph-output', 'figure'),
    [Input('btn-manual-label', 'n_clicks'),
    Input('button-sync-vid', 'n_clicks'),]
)
def updateGraph(btn1, btn2):
    ctx = dash.callback_context
    if not ctx.triggered:
        return dash.no_update
    button_id = ctx.triggered[0]['prop_id'].split('.')[0]

    if button_id == 'btn-manual-label':
        # Update label and confidence data
        start = update_start_time.data
        end = update_end_time.data
        label = update_label.data
        confidence = update_confidence_degree.data

        # Update DataFrame with new label and confidence data
        update_dataframe(df, start, end, label, confidence)

        # Recalculate label indices after updating DataFrame
        labelsStartIndex, labelsEndIndex = calculate_label_indices(df)

        # Replot the graph with updated data
        return plotGraph()

    elif button_id == 'button-sync-vid':
        # Sync video logic
        # Get current video time and offset
        offset = int(update_video_offset.data)
        timestamp = int(update_time.data)

        # Calculate the synchronization point in data
        vid_to_data_sync = calculate_sync_point(df, offset, timestamp)

        # Update the graph with a sync point
        return plotGraph_with_sync_point(vid_to_data_sync)

    else:
        return dash.no_update

def calculate_sync_point(df, offset, timestamp):
    # Convert the first datetime in df to Unix time
    firstDateTime = pd.to_datetime(df['datetime'].iloc[0])
    firstUnixTime = time.mktime(firstDateTime.timetuple())

    # Calculate the synchronization time
    sync_unix_time = firstUnixTime + timestamp - offset

    # Convert back to datetime
    sync_time = pd.to_datetime(sync_unix_time, unit='s', origin='unix')

    # Subtract 5 hours HARD CODED FOR NOW
    adjusted_sync_time = sync_time - timedelta(hours=5)
    formatted_adjusted_sync_time = adjusted_sync_time.strftime('%Y-%m-%d %H:%M:%S')

    return formatted_adjusted_sync_time


def plotGraph_with_sync_point(sync_time):
    global fig_layout

    # Reset and recreate the figure layout
    fig_layout = make_subplots(
        rows=3, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.1,
        row_heights=[5,5,50],
        subplot_titles=("Label Line", "Confidence Line", "Raw Time-Series Data")
    )

    # Add time-series data and label/confidence lines
    for col in cols:
        add_trace_to_layout(fig_layout, df, col, 3, 1)
    update_label_lines(fig_layout, labelsStartIndex, labelsEndIndex, df)
    update_confidence_lines(fig_layout, labelsStartIndex, labelsEndIndex, df)

    # Add sync line
    fig_layout.add_vline(x=sync_time, line_width=2, line_dash="dash", line_color="red")
    fig_layout.add_annotation(x=sync_time, y=0.5, text="Video Sync", showarrow=False, yshift=10)

    # Update layout settings
    fig_layout.update_layout(
        xaxis=dict(),
        xaxis1_rangeslider_visible=False,
        xaxis2_rangeslider_visible=False,
        xaxis3_rangeslider_visible=True,
        height=500,
        legend_title_text="Sensors"
    )

    return fig_layout

def update_dataframe(df, start, end, label, confidence):
    # Ensure the 'datetime' column is in datetime format
    df['datetime'] = pd.to_datetime(df['datetime'])

    # Convert start and end times to datetime
    start = pd.to_datetime(start)
    end = pd.to_datetime(end)

    # Find the closest start and end indices in the DataFrame
    # If exact match is not found, it finds the nearest date
    startIndex = df.index[df['datetime'] >= start][0]
    endIndex = df.index[df['datetime'] <= end][-1]

    # Update label and confidence data in the DataFrame
    df.loc[startIndex:endIndex, 'label'] = label
    df.loc[startIndex:endIndex, 'confidence'] = confidence

# callback for seeking to timestamp in video from user inputted value
@app.callback(
    Output("video-player", "seekTo"), 
    [Input("button-seek-to", "n_clicks"),
    Input('seek-input', 'value')]
)
def set_seekTo(n_clicks, seek_value):
    if 'button-seek-to' == ctx.triggered_id:
        return seek_value

# callback for resetting video UI
@app.callback(
    [Output('video-offset-input', 'value'),Output('seek-input', 'value')],
    [Input('reset-button-2', 'n_clicks')])
def reset_input(n_clicks):
    if n_clicks:
        return '0','0'
    else:
        return '0','0'

# checkbox
@app.callback(Output('start-cb-output', 'children'), [Input('start-checkbox', 'value')])
def start_ui_fill(value):
    if len(value) == 2:
        start_ui_fill.data = value
        return "Start time fill toggled on"
    else:
        start_ui_fill.data = value
        return "Start time fill toggled off"

# fill start UI with clicked data
@app.callback(
    Output('start-input', 'value'),
    Input('graph-output', 'clickData'))
def st_x_value(clickData):
    try:
        if start_ui_fill.data.count('Checked') == 1:
            xval1 = clickData['points'][0]['x']
            st_x_value.data = xval1
            dt_val1 = xval1.split(".")[0]
            return str(dt_val1)
        else:
            return update_start_time.data
    except Exception as e:
        pass

# checkbox
@app.callback(Output('end-cb-output', 'children'), [Input('end-checkbox', 'value')])
def end_ui_fill(value):
    if len(value) == 2:
        end_ui_fill.data = value
        return "End time fill toggled on"
    else:
        end_ui_fill.data = value
        return "End time fill toggled off"

# fill start UI with clicked data
@app.callback(
    Output('end-input', 'value'),
    Input('graph-output', 'clickData'))
def ed_x_value(clickData):
    try:
        if end_ui_fill.data.count('Checked') == 1:
            xval2 = clickData['points'][0]['x']
            ed_x_value.data = xval2
            dt_val2 = xval2.split(".")[0]
            return str(dt_val2)
        else:
            return update_end_time.data
    except Exception as e:
        pass

# run app in jupyter mode externally
app.run_server(jupyter_mode='external')

Dash app running on http://127.0.0.1:8050/


firstDateTime 2021-10-08 16:50:21
firstUnixTime 1633729821.0
sync_unix_time 1633729821.0
sync_time 2021-10-08 21:50:21
adjusted_sync_time 2021-10-08 16:50:21
firstDateTime 2021-10-08 16:50:21
firstUnixTime 1633729821.0
sync_unix_time 1633730077.0
sync_time 2021-10-08 21:54:37
adjusted_sync_time 2021-10-08 16:54:37
